# Barcodes

Show that barcodes don't matter and that we can pick out when they do.

In [ ]:
%load_ext autoreload
%autoreload 2  
%reload_ext autoreload
import sys
sys.path.append('../../src')
from util import *
from config import *
from study_gen import study
import generate_dataset
import ipynbname
import plots

### Barcodes comparison scatter plot
- x-axis = barcode1 
- y-axes = barcode2
- fit: 
    - x=y (line of identity)
    - best-fit line
    - Pearson R
- if more than 2 barcode reps then heatmap pearson R everything vs. everything 

In [ ]:
## Change sample ##
sample = '01_02_S23_reads'   
##############################    

plot = plots.barcode_comparison_scatter_plot(study, sample)

save_plotly_fig(
    ipynbname.path(), 
    '[A] Barcodes comparison scatter plot/{}'.format(sample),
    plot['fig']
    )

plot['fig'].show()

### Combined barcode reps in sample
- x-axis = barcode1/barcode2 names 
- y-axis = pearson R 
- ranked by R
- Or is there a better way to do this–% percent total??


In [ ]:
plot = plots.combined_barcode_replicates_in_sample(study)
save_plotly_fig(ipynbname.path(), '[B] Combined barcode reps in sample', plot['fig'])
plot['fig'].show()